## Práctica 1: Minería de textos

### Preparativos

In [62]:
!pip install spacy


In [63]:
!python -m spacy download es_core_news_sm
!pyhon -m spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 29.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
/bin/bash: line 1: pyhon: command not found


In [64]:
import os
os.listdir()

['.config',
 'texto_manual_plano.txt',
 '__pycache__',
 'true_labels2.txt',
 'TESTB-ESP.txt',
 '.ipynb_checkpoints',
 'sample_data']

### Extraer texto plano

In [76]:
import re

def extract_plain_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    text = []
    for line in lines:
        if line.strip():
            word = line.split()[0]  # Toma la primera columna (palabra)
            text.append(word)
        else:
            text.append('')  # Si la línea está vacía es espacio

    # Unimos las palabras con espacios
    plain_text = ' '.join(text)
    return plain_text

In [77]:
# Convertimos y guardamos el texto plano
plain_text = extract_plain_text('TESTB-ESP.txt')
with open('plain_text.txt', 'w', encoding='utf-8') as out_file:
    out_file.write(plain_text)

In [78]:
#Esta fucnión la utilizamos para tener el output de spacy en el mismo formato que el documento TESTB-ESP
def text_to_iob(doc):
    iob_output = []
    for token in doc:
        # Si el token está dentro de una entidad nombrada (B, I, O)
        if token.ent_iob_ == "B":
            iob_output.append(f"{token.text} B-{token.ent_type_}")
        elif token.ent_iob_ == "I":
            iob_output.append(f"{token.text} I-{token.ent_type_}")
        elif token.text!=' ':
            # Si no está dentro de una entidad nombrada, es Outside (O)
            iob_output.append(f"{token.text} O")
        else:
            iob_output.append(' ')

    iob_output = "\n".join(iob_output)

    return iob_output

### Parte obligatoria

In [68]:
##Customizamos la tockenización y cargamos un modelo preentrenado
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("es_core_news_sm")
nlp.tokenizer=Tokenizer(nlp.vocab)

In [69]:
# Leer el texto desde un archivo
with open("plain_text.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Procesar el texto con SpaCy
doc = nlp(text)

In [70]:
print(dir(doc))

['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '_bulk_merge', '_context', '_get_array_attrs', '_realloc', '_vector', '_vector_norm', 'cats', 'char_span', 'copy', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_dict', 'from_disk', 'from_docs', 'from_json', 'get_extension', 'get_lca_matrix', 'has_annotation', 'has_extension', 'has_unknown_spaces', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'noun_chunks', 'noun_chunks_iterator', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_ents', 'set_extension', 'similarity', 'spans', 'tensor', 'text', 'te

In [71]:
spacy_otput=text_to_iob(doc)
with open('pred_labels.txt', 'w', encoding='utf-8') as out_file:
    out_file.write(spacy_otput)

In [72]:
def combine (true_labels_file, pred_labels_file, output_file):
    """
    Combina las etiquetas reales y predichas en un único archivo en formato CoNLL.

    Args:
        true_labels_file (str): Ruta del archivo con las etiquetas reales.
        pred_labels_file (str): Ruta del archivo con las etiquetas predichas.
        output_file (str): Ruta del archivo de salida combinado.

    Returns:
        None
    """
    # Leemos ambos archivos
    with open(true_labels_file, "r", encoding="utf-8") as f:
        true_lines = f.read().strip().splitlines()

    with open(pred_labels_file, "r", encoding="utf-8") as f:
        pred_lines = f.read().strip().splitlines()

    # Creamos el archivo combinado
    with open(output_file, "w", encoding="utf-8") as f:
        for true_line, pred_line in zip(true_lines, pred_lines):
            if true_line.strip() == "" or pred_line.strip() == "":
                # Línea vacía (separación de oraciones)
                f.write("\n")
                continue

            # Dividir palabras y etiquetas
            true_word, true_label = true_line.rsplit(maxsplit=1)
            pred_word, pred_label = pred_line.rsplit(maxsplit=1)

            f.write(f"{true_word} {true_label} {pred_label}\n")

        f.write("\n")


In [73]:

# Nombres de los archivos
true_labels_file = "TESTB-ESP.txt"
pred_labels_file = "pred_labels.txt"
output_file = "combined.txt"
combine(true_labels_file,pred_labels_file,output_file)

In [74]:
# Importar el script
import conlleval
from conlleval import evaluate_conll_file

with open("combined.txt", "r") as file:
    result = evaluate_conll_file(file)

print(result)

processed 53050 tokens with 3559 phrases; found: 3945 phrases; correct: 2047.
accuracy:  56.23%; (non-O)
accuracy:  92.23%; precision:  51.89%; recall:  57.52%; FB1:  54.56
              LOC: precision:  51.20%; recall:  71.03%; FB1:  59.51  1504
             MISC: precision:  12.36%; recall:  23.53%; FB1:  16.21  647
              ORG: precision:  75.88%; recall:  44.50%; FB1:  56.10  821
              PER: precision:  58.99%; recall:  78.10%; FB1:  67.21  973
(51.88846641318124, 57.516156223658335, 54.55756929637528)


### Parte opcional

In [75]:
nlp2 = spacy.load("es_core_news_sm")
nlp2.tokenizer=Tokenizer(nlp2.vocab)
with open("texto_manual_plano.txt", "r", encoding="utf-8") as file:
    text2 = file.read()

# Procesar el texto con SpaCy
doc2 = nlp2(text2)
#Guardamos la clasificación predicha enun documento pred_labels2.txt
spacy_output2=text_to_iob(doc2)
with open('pred_labels2.txt', 'w', encoding='utf-8') as out_file:
    out_file.write(spacy_output2)

# Creamos el documento en formato word gold pred
true_labels_file = "true_labels2.txt"
pred_labels_file = "pred_labels2.txt"
output_file = "combined2.txt"
combine(true_labels_file,pred_labels_file,output_file)

#Sacamos las métricas para evaluar el resultado final
with open("combined2.txt", "r") as file:
    result = evaluate_conll_file(file)

print(result)

processed 690 tokens with 50 phrases; found: 62 phrases; correct: 7.
accuracy:  10.89%; (non-O)
accuracy:  73.77%; precision:  11.29%; recall:  14.00%; FB1:  12.50
              LOC: precision:  18.18%; recall:  25.00%; FB1:  21.05  11
             MISC: precision:   8.70%; recall:  40.00%; FB1:  14.29  23
              ORG: precision:  10.53%; recall:   7.14%; FB1:   8.51  19
              PER: precision:  11.11%; recall:  11.11%; FB1:  11.11  9
(11.29032258064516, 14.000000000000002, 12.5)
